In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# sample text: it is to be read from file system later on... dataset is supposed to have ~ 1 mil. characters
text = open('./corpus/corpus.txt', 'r').read()

In [3]:
# some preprocessing required...
#like take out the capitals, do something with the fullstops, indents, quotes etc.

In [4]:
chars = list(set(text))
vocab_size = len(chars)

In [5]:
char_to_index = { c:i for i,c in enumerate(chars) }
index_to_char = { i:c for i,c in enumerate(chars) }

In [6]:
def one_hot(char_string):
    sentence = np.empty([0, len(chars)])
    for char in char_string:
        vector = np.zeros((1, len(chars)))
        vector[0][char_to_index[char]] = 1
        sentence = np.append(sentence, vector, axis=0)
    return sentence

def one_hot_char(char):
    vector = np.zeros(len(chars))
    vector[char_to_index[char]] = 1
    return vector

In [7]:
# generate dataset X and y
chars_per_sequence = 50
skip = 2

#X = np.empty([0, chars_per_sequence, len(chars)])
X_train = []
end_char = []
for i in range(0, len(text) - chars_per_sequence-1, skip):
    X_train.append(one_hot(text[i:i+chars_per_sequence]))
    end_char.append(one_hot_char(text[i+chars_per_sequence]))
X_train = np.array(X_train)
end_char = np.array(end_char)

In [8]:
#define model parameters

hidden_state_size = 1024
state_size = 512
i_size = vocab_size
o_size = vocab_size


batch_size = 42
sequence_length = chars_per_sequence#

In [13]:
## construct model graph and rout inputs, outputs
# prefixing the None of the batch size...
graph = tf.Graph()
with graph.as_default():
    
    w_ii = tf.Variable(tf.random.truncated_normal([i_size, hidden_state_size]))
    w_io = tf.Variable(tf.random.truncated_normal([hidden_state_size, hidden_state_size]))
    b_i = tf.Variable(tf.random.truncated_normal([hidden_state_size]))
    
    w_oi = tf.Variable(tf.random.truncated_normal([i_size, hidden_state_size]))
    w_oo = tf.Variable(tf.random.truncated_normal([hidden_state_size, hidden_state_size]))
    b_o = tf.Variable(tf.random.truncated_normal([hidden_state_size]))

    w_mi = tf.Variable(tf.random.truncated_normal([i_size, hidden_state_size]))
    w_mo = tf.Variable(tf.random.truncated_normal([hidden_state_size, hidden_state_size]))
    b_m = tf.Variable(tf.random.truncated_normal([hidden_state_size]))
    
    w_fi = tf.Variable(tf.random.truncated_normal([i_size, hidden_state_size]))
    w_fo = tf.Variable(tf.random.truncated_normal([hidden_state_size, hidden_state_size]))
    b_f = tf.Variable(tf.random.truncated_normal([hidden_state_size]))    
    
    w_out = tf.Variable(tf.random.truncated_normal([hidden_state_size, vocab_size]))
    b_out = tf.Variable(tf.random.truncated_normal([vocab_size])) 
    
    def lstm_cell(i, o, state):
        input_gate = tf.sigmoid(i @ w_ii + o @ w_io  + b_i);
        output_gate = tf.sigmoid(i @ w_oi + o @ w_oo + b_o);
        forget_gate = tf.sigmoid(i @ w_fi + o @ w_fo + b_f);
        memory_gate = tf.tanh(i @ w_mi + o @ w_mo + b_m);
        
        state = forget_gate * state + input_gate * memory_gate 
        output = output_gate * tf.tanh(state)
        
        return output, state

    X = tf.placeholder(dtype=tf.float32, shape=[batch_size, sequence_length, vocab_size])
    last_char = tf.placeholder(dtype=tf.float32, shape=[batch_size, vocab_size])

    output_sequence = []
    y = []

    state = tf.Variable(tf.zeros([batch_size, hidden_state_size]))
    o_vector = tf.Variable(tf.zeros([batch_size, hidden_state_size]))

    for i in range(sequence_length):
        o_vector, state = lstm_cell(X[:, i, :], o_vector, state)
        output_sequence.append(o_vector)
        if i < sequence_length-1:
            y.append(X[:, i+1, :]) 
        else:
            y.append(last_char)
    output_sequence = tf.stack(output_sequence, axis=1)
    y = tf.stack(y, axis=1)
    
    chars_output = tf.reshape(output_sequence, shape=[-1, hidden_state_size]) @ w_out + b_out
    chars_output = tf.reshape(chars_output, shape=[batch_size, sequence_length, vocab_size])
    
    
    loss = sequence_length*tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=chars_output, labels=y) )

    #optimizer = tf.train.AdamOptimizer()
    optimizer = tf.train.GradientDescentOptimizer(0.001)
    step = optimizer.minimize(loss)
    
    
    #generate text character by character:
    o_vector_test = tf.placeholder(dtype=tf.float32, shape=[hidden_state_size])
    state_test = tf.placeholder(dtype=tf.float32, shape=[hidden_state_size])
    x_test = tf.placeholder(dtype=tf.float32, shape=[vocab_size])
    o_vector_test_out, state_test_out = lstm_cell(tf.reshape(x_test, shape=[1, -1]), tf.reshape(o_vector_test, shape=[1, -1]), state_test)
    output_char_test = tf.argmax(state_test_out @ w_out + b_out, axis=1)
    

In [48]:
gpuconfig = tf.GPUOptions(per_process_gpu_memory_fraction=0.33)
with tf.Session(graph=graph, config=tf.ConfigProto(gpu_options=gpuconfig)) as sess:
    offs = 0
    max_steps = 1000
    saver = tf.train.Saver()
    tf.global_variables_initializer().run()
    for i in range(max_steps):
        si = (i+offs)%X_train.shape[0]
        if si+batch_size < X_train.shape[0]:
            X_batch = X_train[si:si+batch_size]
            end_char_batch = end_char[si:si+batch_size]
        else:
            offs_new = (si+batch_size)%X_train.shape[0]
            X_batch = np.concatenate([X_train[si:], X_train[:offs_new]])
            end_char_batch = np.concatenate([end_char[si:], end_char[:offs_new]])
            #offs = offs_new
        
        k = sess.run(step, { X: X_batch, last_char: end_char_batch})
        if(i%10 == 0):
            print(i, sess.run(loss, { X: X_batch, last_char: end_char_batch}))
    saver.save(sess, './model/model.ckpt' )

0 1081.3002
10 996.949
20 883.97076
30 770.5477
40 712.9703
50 684.7426
60 681.9686
70 664.63934
80 661.029
90 624.83167
100 623.3298
110 645.6527
120 641.8442
130 649.7263
140 654.1144
150 640.8444
160 634.3124
170 624.72095
180 589.7375
190 595.03534
200 575.44415
210 582.74817
220 592.5355
230 610.99536
240 624.6385
250 621.167
260 626.63434
270 617.5976
280 600.82605
290 599.416
300 603.4523
310 607.79456
320 633.5041
330 604.64
340 591.89484
350 583.9649
360 594.98096
370 597.11127
380 584.3137
390 576.55426
400 548.5362
410 542.6016
420 539.61743
430 535.68097
440 541.1704
450 545.51886
460 544.907
470 545.02344
480 542.2078
490 566.48755
500 551.5916
510 542.8602
520 539.93976
530 507.38763
540 506.52185
550 502.51965
560 509.30118
570 510.08987
580 501.45847
590 507.28693
600 493.88104
610 502.1738
620 513.8662
630 514.1284
640 506.58264
650 519.37616
660 498.40887
670 487.97446
680 463.6243
690 445.2973
700 455.08752
710 459.8486
720 451.32352
730 486.50082
740 509.41534
750 5

In [49]:
tf.reset_default_graph()
with tf.Session(graph=graph) as sess:
#    model = tf.train.load_checkpoint('./model/model.ckpt')
    saver = tf.train.Saver()
    saver.restore(sess, './model/model.ckpt')
    z = ['a', np.zeros([hidden_state_size]), np.zeros([hidden_state_size])]

    
    output_string = z[0]
    for i in range(20):
        z = sess.run([output_char_test,o_vector_test_out, state_test_out], {x_test: one_hot_char(z[0]), o_vector_test: z[1],\
                                                                        state_test: z[2] } )
    
        z[0] = index_to_char[z[0][0]]
        z[1], z[2] = z[1][0], z[2][0]

        output_string += z[0]
    

INFO:tensorflow:Restoring parameters from ./model/model.ckpt


In [50]:
output_string

'adeseeemmmmsslldldldl'